In [1]:
import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Concatenate, Dropout, Subtract, \
                        Flatten, MaxPooling2D, Multiply, Lambda, Add, Dot
from keras.backend import constant
from keras import optimizers

from keras.engine.topology import Layer ##idea is that first use only loc vol until first maturity and minimize loss
from keras.models import Model           ## function. Then fix weights for this first part and then try to minimize 
from keras.layers import Input          ## loss function for options with second maturity. Then fix these weights
from keras import initializers         ## and continue. We need different training data input and output for every try
from keras.constraints import max_norm ## maybe better to have a dictionary of maturities mapping to list of tuples
import matplotlib.pyplot as plt       ## consisting of strike price and option prices
import keras.backend as K             ## then only need to hedge the ones with this maturity
                                       ## if never use NN then don't need to set layers not trainable, it wont matter
                                    ## probably only need to set NN of previous local vol to not trainable
                                    ## just have a list of layers and put the two previous ones not trainable
                                    ## and add two new layers. Need to add new model since have new comp. graph
                                    ## definera model, träna och fortsätt så. Att ha en hedge NN för varje option
                                    ## är typ samma som ha en för varje maturity och ta strike som input

Using TensorFlow backend.


In [2]:
def find_next(list,value):
    index = 0
    for ind,val in enumerate(list):
        if val >= value:
            index = ind
            break
    return index

def locvol(t,S):
    return np.sin(S)**2+t 

def stochasticvol(t):
    return 1

def optionprice(strikep,expiration,spotp): ##can possibly make much faster by implementing in tensorflow
    N = 50 #time discretization
    iterations = 10**5
    dt = expiration/N
    mat = np.random.normal(0,np.sqrt(dt),(iterations,N))
    
    sum = 0
    for row in mat:
        stockp = spotp
        for i in range(N):
            stockp += stochasticvol(i*dt)*locvol(i*dt,stockp)*row[i]
        sum += np.maximum(stockp-strikep,0)
    return sum/iterations
            

In [3]:
N = 40 # time discretization (should fit to maturities)!!!
maturities = [0.5,1.0] #should be sorted list
T = 1.0
dt = T/N #time discretization
layers = []

option1 = optionprice(0.9,0.5,1)
option2 = optionprice(0.9,1.0,1)
option3 = optionprice(1.0,0.5,1)
option4 = optionprice(1.0,1.0,1)
option5 = optionprice(1.1,0.5,1)
option6 = optionprice(1.1,1.0,1)

P = {(0.9,0.5): option1,
     (0.9,1.0): option2,
     (1.0,0.5): option3,
     (1.0,1.0): option4,
     (1.1,0.5): option5,
     (1.1,1.0): option6}
Pmat = {}
Pmat[0.5] = [(0.9,option1),(1.0,option3),(1.1,option5)] 
Pmat[1.0] = [(0.9,option2),(1.0,option4),(1.1,option6)]
    
normal = tf.distributions.Normal(loc=0., scale=1.)

def BS(x):
    price=x[0]
    vola=x[1]
    return normal.cdf((K.log(K.abs(price)/key[0])-0.5*(key[1]-j*T/N)*vola**2)/(0.0001+np.sqrt(key[1]-j*T/N)*vola))
# increases computational time

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


{(0.9, 0.5): 0.30314683276564763,
 (0.9, 1.0): 0.4896398480122337,
 (1.0, 0.5): 0.26043877511145946,
 (1.0, 1.0): 0.4502535778828075,
 (1.1, 0.5): 0.22357277457073244,
 (1.1, 1.0): 0.40751712389607087}

In [23]:

hedges = {}
for mat in maturities:
    hedgeLayers =[]
    for j in range(int(mat*N/T)):   #T/N*steps=maturity so steps=maturity*N/T
        hedge = Dense(1, activation='tanh', trainable=True,
                      kernel_initializer=initializers.RandomNormal(0,0.1),
                      bias_initializer='random_normal')
        hedgeLayers = hedgeLayers+[hedge]
    hedges[mat] = hedgeLayers

layer1 = Dense(10, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layer2 = Dense(1, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layers = layers+[[layer1,layer2]]

price = Input(shape=(1,))
hedgepf = [Input(shape=(1,)) for i in range(len(Pmat[0.5]))]
strikes = [Input(shape=(1,)) for i in range(len(Pmat[0.5]))]
inputs = [price] + hedgepf+strikes
inputshelper = []
pricekey = [tf.constant(0) for i in range(len(Pmat[0.5]))]


for j in range(0,20):    
    ind = find_next(maturities,j*dt)
    helper0 = layers[ind][0](price) 
    helper0 = layers[ind][1](helper0)
    BMincr = Input(shape=(1,))      
    stochvol = Input(shape=(1,))
    inputshelper = inputshelper + [stochvol]
    inputs = inputs + [BMincr]
    helper = Multiply()([helper0,BMincr])  
    priceincr = Multiply()([helper,stochvol])
    
    for i,tup in enumerate(Pmat[0.5]):
        strikePrice = strikes[i]
        optionPrice = tup[1]
        key = (strikePrice,0.5)
        
        BSstrategy = Lambda(BS)([price,helper0])
        l = Concatenate()([price,strikePrice])
        hedgeratio = hedges[mat][j](l)
        hedgeratio = Add()([hedgeratio,BSstrategy])
        hedge = Multiply()([priceincr,hedgeratio])
        hedgepf[i] = Add()([hedgepf[i],hedge])
        if j == 19:
            helper2 = Lambda(lambda x : 0.5*(abs(x-strikePrice)+x-strikePrice))(price)
            helper2 = Subtract()([helper2,hedgepf[i]])
            pricekey[i] = helper2 
    price = Add()([price,priceincr]) ##change order here??
    

inputs = inputs + inputshelper
pricekey = Concatenate()(pricekey)
model1 = Model(inputs=inputs, outputs=pricekey)   

Ltrain = 10**5
xtrain =([np.ones(Ltrain)] + [np.zeros(Ltrain) for i in range(len(Pmat[0.5]))]+
         [np.ones(Ltrain)*Pmat[0.5][i][0] for i in range(len(Pmat[0.5]))]+
         [np.random.normal(0,np.sqrt(dt),Ltrain) for i in range(20)]+
         [np.ones(Ltrain)*stochasticvol(i*dt) for i in range(20)])

ytrain=np.zeros((Ltrain,len(Pmat[0.5])))
for i in range(Ltrain):
    for j,tup in enumerate(Pmat[0.5]):
        optionPrice = tup[1]
        ytrain[i,j]= optionPrice

model1.compile(optimizer='adam', 
              loss='mean_squared_error')

model1.fit(x=xtrain,y=ytrain, epochs=20,verbose=True)
print(np.mean(model1.predict(xtrain),axis=0))


Epoch 1/20
100000/100000 [==============================] - 55s 551us/step - loss: 0.0052
Epoch 2/20
100000/100000 [==============================] - 43s 432us/step - loss: 0.0033
Epoch 3/20
 14016/100000 [===>..........................] - ETA: 37s - loss: 0.0030

KeyboardInterrupt: 

[(0.9, 0.3040138138478351), (1.0, 0.2590291563267426), (1.1, 0.21983807359944546)]


In [21]:

layer1 = Dense(10, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layer2 = Dense(1, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layers = layers+[[layer1,layer2]]

price = Input(shape=(1,))
hedgepf = [Input(shape=(1,)) for i in range(len(Pmat[1.0]))]
strikes = [Input(shape=(1,)) for i in range(len(Pmat[1.0]))]
inputs = [price] + hedgepf+strikes
inputshelper = []
pricekey = [tf.constant(0) for i in range(len(Pmat[1.0]))]


for j in range(0,40):    
    ind = find_next(maturities,j*dt)
    if ind < 20:
        layers[ind][0].trainable=False
        layers[ind][1].trainable=False
    helper0 = layers[ind][0](price) 
    helper0 = layers[ind][1](helper0)
    BMincr = Input(shape=(1,))      
    stochvol = Input(shape=(1,))
    inputshelper = inputshelper + [stochvol]
    inputs = inputs + [BMincr]
    helper = Multiply()([helper0,BMincr])  
    priceincr = Multiply()([helper,stochvol])
    
    for i,tup in enumerate(Pmat[1.0]):
        strikePrice = strikes[i]
        optionPrice = tup[1]
        key = (strikePrice,1.0)
        
        BSstrategy = Lambda(BS)([price,helper0])
        l = Concatenate()([price,strikePrice])
        hedgeratio = hedges[mat][j](l)
        hedgeratio = Add()([hedgeratio,BSstrategy])
        hedge = Multiply()([priceincr,hedgeratio])
        hedgepf[i] = Add()([hedgepf[i],hedge])
        if j == 39:
            helper2 = Lambda(lambda x : 0.5*(abs(x-strikePrice)+x-strikePrice))(price)
            helper2 = Subtract()([helper2,hedgepf[i]])
            pricekey[i] = helper2 
    price = Add()([price,priceincr]) ##change order here??
    

inputs = inputs + inputshelper
pricekey = Concatenate()(pricekey)
model2 = Model(inputs=inputs, outputs=pricekey)   
print(model2.summary())

xtrain =([np.ones(Ltrain)] + [np.zeros(Ltrain) for i in range(len(Pmat[1.0]))]+
         [np.ones(Ltrain)*Pmat[1.0][i][0] for i in range(len(Pmat[1.0]))]+
         [np.random.normal(0,np.sqrt(dt),Ltrain) for i in range(40)]+
         [np.ones(Ltrain)*stochasticvol(i*dt) for i in range(40)])

ytrain=np.zeros((Ltrain,len(Pmat[1.0])))
for i in range(Ltrain):
    for j,tup in enumerate(Pmat[1.0]):
        optionPrice = tup[1]
        ytrain[i,j]= optionPrice

model1.compile(optimizer='adam', 
              loss='mean_squared_error')

model1.fit(x=xtrain,y=ytrain, epochs=15,verbose=True)
print(np.mean(model1.predict(xtrain),axis=0))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_612 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
dense_123 (Dense)               (None, 10)           20          input_612[0][0]                  
                                                                 add_1827[0][0]                   
                                                                 add_1834[0][0]                   
                                                                 add_1841[0][0]                   
                                                                 add_1848[0][0]                   
                                                                 add_1855[0][0]                   
          

                                                                 add_1935[0][0]                   
__________________________________________________________________________________________________
add_1942 (Add)                  (None, 1)            0           dense_596[16][0]                 
                                                                 lambda_864[0][0]                 
__________________________________________________________________________________________________
lambda_867 (Lambda)             (None, 1)            0           add_1946[0][0]                   
                                                                 dense_124[206][0]                
__________________________________________________________________________________________________
concatenate_850 (Concatenate)   (None, 2)            0           add_1953[0][0]                   
                                                                 input_617[0][0]                  
__________

Epoch 1/15
10000/10000 [==============================] - 28s 3ms/step - loss: 0.1253
Epoch 2/15
10000/10000 [==============================] - 9s 868us/step - loss: 0.1040
Epoch 3/15
10000/10000 [==============================] - 9s 862us/step - loss: 0.0937
Epoch 4/15
10000/10000 [==============================] - 8s 820us/step - loss: 0.0852
Epoch 5/15
10000/10000 [==============================] - 9s 909us/step - loss: 0.0776
Epoch 6/15
10000/10000 [==============================] - 9s 900us/step - loss: 0.0710
Epoch 7/15
10000/10000 [==============================] - 9s 873us/step - loss: 0.0653
Epoch 8/15
10000/10000 [==============================] - 9s 877us/step - loss: 0.0602
Epoch 9/15
10000/10000 [==============================] - 9s 881us/step - loss: 0.0559
Epoch 10/15
10000/10000 [==============================] - 9s 891us/step - loss: 0.0522
Epoch 11/15
10000/10000 [==============================] - 9s 885us/step - loss: 0.0489
Epoch 12/15
10000/10000 [=================

[(0.9, 0.4883143250160542), (1.0, 0.4539207056510781), (1.1, 0.40772288733728274)]


In [50]:







Ltrain = 10**4
xtrain =([np.ones(Ltrain)] + [np.zeros(Ltrain) for i in range(len(Pmat[0.5]))]+
         [np.random.normal(0,np.sqrt(dt),Ltrain) for i in range(20)]+
         [np.ones(Ltrain)*stochasticvol(i*dt) for i in range(20)])

ytrain=np.zeros((Ltrain,len(Pmat[0.5])))
for i in range(Ltrain):
    for j,tup in enumerate(Pmat[0.5]):
        optionPrice = tup[1]
        ytrain[i,j]= optionPrice

model1_trainhedge.compile(optimizer='adam', 
              loss='mean_squared_error')
model1_trainloc.compile(optimizer='adam', 
              loss='mean_squared_error')
for i in range(2):
    model1_trainloc.fit(x=xtrain,y=ytrain, epochs=5,verbose=True)
    y = model1_trainloc.get_weights()
    model1_trainhedge.set_weights(y)
      
    
    model1_trainhedge.fit(x=xtrain,y=ytrain, epochs=5,verbose=True)
    x = model1_trainhedge.get_weights()
    model1_trainloc.set_weights(x) 
    
    print(np.mean(model1_trainhedge.predict(xtrain),axis=0))


Epoch 1/5
10000/10000 [==============================] - 85s 8ms/step - loss: 0.0197
Epoch 2/5
10000/10000 [==============================] - 15s 1ms/step - loss: 0.0196
Epoch 3/5
10000/10000 [==============================] - 18s 2ms/step - loss: 0.0196
Epoch 4/5
10000/10000 [==============================] - 14s 1ms/step - loss: 0.0196
Epoch 5/5
10000/10000 [==============================] - 14s 1ms/step - loss: 0.0196
Epoch 1/5
10000/10000 [==============================] - 91s 9ms/step - loss: 0.0193
Epoch 2/5
10000/10000 [==============================] - 17s 2ms/step - loss: 0.0188
Epoch 3/5
10000/10000 [==============================] - 17s 2ms/step - loss: 0.0184
Epoch 4/5
10000/10000 [==============================] - 17s 2ms/step - loss: 0.0180
Epoch 5/5
10000/10000 [==============================] - 18s 2ms/step - loss: 0.0176
[0.20254576 0.1723781  0.145039  ]
Epoch 1/5
10000/10000 [==============================] - 15s 2ms/step - loss: nan
Epoch 2/5
10000/10000 [==========

KeyboardInterrupt: 

In [54]:
print(model1_trainhedge.summary())
print(model1_trainloc.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1717 (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
dense_377 (Dense)               (None, 10)           20          input_1717[0][0]                 
                                                                 add_5467[0][0]                   
                                                                 add_5474[0][0]                   
                                                                 add_5481[0][0]                   
                                                                 add_5488[0][0]                   
                                                                 add_5495[0][0]                   
          

__________________________________________________________________________________________________
multiply_1341 (Multiply)        (None, 1)            0           dense_378[109][0]                
                                                                 input_589[0][0]                  
__________________________________________________________________________________________________
input_590 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
multiply_1342 (Multiply)        (None, 1)            0           multiply_1341[0][0]              
                                                                 input_590[0][0]                  
__________________________________________________________________________________________________
add_1883 (Add)                  (None, 1)            0           add_1876[0][0]                   
          

lambda_879 (Lambda)             (None, 1)            0           add_1953[0][0]                   
__________________________________________________________________________________________________
add_1959 (Add)                  (None, 1)            0           add_1952[0][0]                   
                                                                 multiply_1400[0][0]              
__________________________________________________________________________________________________
subtract_37 (Subtract)          (None, 1)            0           lambda_875[0][0]                 
                                                                 add_1955[0][0]                   
__________________________________________________________________________________________________
subtract_38 (Subtract)          (None, 1)            0           lambda_877[0][0]                 
                                                                 add_1957[0][0]                   
__________

In [45]:


hedges = {}
for key in P.keys():
    hedgeLayers =[]
    for j in range(int(key[1]*N/T)):   #T/N*steps=maturity so steps=maturity*N/T
        hedge = Dense(1, activation='tanh', trainable=True,
                      kernel_initializer=initializers.RandomNormal(0,0.1),
                      bias_initializer='random_normal')
        hedgeLayers = hedgeLayers+[hedge]
    hedges[key] = hedgeLayers

layer1 = Dense(10, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layer2 = Dense(1, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layers = layers+[[layer1,layer2]]

price = Input(shape=(1,))
hedgepf = [Input(shape=(1,)) for i in range(len(Pmat[0.5]))]
inputs = [price] + hedgepf
inputshelper = []
pricekey = [tf.constant(0) for i in range(len(Pmat[0.5]))]

for j in range(0,20):    
    ind = find_next(maturities,j*dt)
    helper0 = layers[ind][0](price) 
    layers[ind][0].trainable=True
    layers[ind][1].trainable=True
    helper0 = layers[ind][1](helper0)
    BMincr = Input(shape=(1,))      
    stochvol = Input(shape=(1,))
    inputshelper = inputshelper + [stochvol]
    inputs = inputs + [BMincr]
    helper = Multiply()([helper0,BMincr])  
    priceincr = Multiply()([helper,stochvol])
    
    for i,tup in enumerate(Pmat[0.5]):
        strikePrice = tup[0]
        optionPrice = tup[1]
        key = (strikePrice,0.5)
        
        BSstrategy = Lambda(BS)([price,helper0])
        hedgeratio = hedges[key][j](price)
        hedgeratio = Add()([hedgeratio,BSstrategy])
        hedge = Multiply()([priceincr,hedgeratio])
        hedgepf[i] = Add()([hedgepf[i],hedge])
        if j == 19:
            helper2 = Lambda(lambda x : 0.5*(abs(x-strikePrice)+x-strikePrice))(price)
            helper2 = Subtract()([helper2,hedgepf[i]])
            pricekey[i] = helper2 
    price = Add()([price,priceincr]) ##change order here??
    

inputs = inputs + inputshelper
pricekey = Concatenate()(pricekey)
model1 = Model(inputs=inputs, outputs=pricekey)    

Ltrain = 10**4
xtrain =([np.ones(Ltrain)] + [np.zeros(Ltrain) for i in range(len(Pmat[0.5]))]+
         [np.random.normal(0,np.sqrt(dt),Ltrain) for i in range(20)]+
         [np.ones(Ltrain)*stochasticvol(i*dt) for i in range(20)])

ytrain=np.zeros((Ltrain,len(Pmat[0.5])))
for i in range(Ltrain):
    for j,tup in enumerate(Pmat[0.5]):
        optionPrice = tup[1]
        ytrain[i,j]= optionPrice

model1.compile(optimizer='adam', 
              loss='mean_squared_error')

model1.fit(x=xtrain,y=ytrain, epochs=10,verbose=True)

print(np.mean(model1_trainhedge.predict(xtrain),axis=0))

print(model1.summary())

Epoch 1/10
10000/10000 [==============================] - 87s 9ms/step - loss: 0.0435
Epoch 2/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.0257
Epoch 3/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.0225
Epoch 4/10
10000/10000 [==============================] - 17s 2ms/step - loss: 0.0214
Epoch 5/10
10000/10000 [==============================] - 17s 2ms/step - loss: 0.0213
Epoch 6/10
10000/10000 [==============================] - 17s 2ms/step - loss: 0.0206
Epoch 7/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.0200
Epoch 8/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.0195
Epoch 9/10
10000/10000 [==============================] - 17s 2ms/step - loss: 0.0190
Epoch 10/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.0190
[0.21572204 0.18379135 0.15471205]
__________________________________________________________________________________________________
Layer

In [46]:
print(Pmat[0.5])

[(0.9, 0.30307521710427626), (1.0, 0.2594805746524422), (1.1, 0.22182572524237912)]


In [23]:
layers[-1][0].trainable=False
layers[-1][1].trainable=False
layer1 = Dense(10, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layer2 = Dense(1, activation='tanh', trainable=True,
              kernel_initializer=initializers.RandomNormal(0,1),
              bias_initializer='random_normal')
layers = layers+[[layer1,layer2]]

price = Input(shape=(1,))
hedgepf = [Input(shape=(1,)) for i in range(len(Pmat[1.0]))]
inputs = [price] + hedgepf
inputshelper = []
pricekey = [tf.constant(0) for i in range(len(Pmat[1.0]))]

for j in range(0,40):    
    ind = find_next(maturities,j*dt)
    helper0 = layers[ind][0](price) 
    helper0 = layers[ind][1](helper0)
    BMincr = Input(shape=(1,))      
    stochvol = Input(shape=(1,))
    inputshelper = inputshelper + [stochvol]
    inputs = inputs + [BMincr]
    helper = Multiply()([helper0,BMincr])  
    priceincr = Multiply()([helper,stochvol])
    
    for i,tup in enumerate(Pmat[1.0]):
        strikePrice = tup[0]
        optionPrice = tup[1]
        key = (strikePrice,1.0)
        
        BSstrategy = Lambda(BS)([price,helper0])
        hedgeratio = hedges[key][j](price)
        hedgeratio = Add()([hedgeratio,BSstrategy])
        hedge = Multiply()([priceincr,hedgeratio])
        hedgepf[i] = Add()([hedgepf[i],hedge])
        if j == 39:
            helper2 = Lambda(lambda x : 0.5*(abs(x-strikePrice)+x-strikePrice))(price)
            helper2 = Subtract()([helper2,hedgepf[i]])
            pricekey[i] = helper2 
    price = Add()([price,priceincr]) ##change order here??
    

inputs = inputs + inputshelper
pricekey = Concatenate()(pricekey)
model1 = Model(inputs=inputs, outputs=pricekey)    

Ltrain = 5*10**3
xtrain =([np.ones(Ltrain)] + [np.zeros(Ltrain) for i in range(len(Pmat[1.0]))]+
         [np.random.normal(0,np.sqrt(dt),Ltrain) for i in range(40)]+
         [np.ones(Ltrain)*stochasticvol(i*dt) for i in range(40)])



ytrain=np.zeros((Ltrain,len(Pmat[1.0])))
for i in range(Ltrain):
    for j,tup in enumerate(Pmat[1.0]):
        optionPrice = tup[1]
        ytrain[i,j]= optionPrice
        
model1.compile(optimizer='adam', 
              loss='mean_squared_error')
model1.fit(x=xtrain,y=ytrain,epochs=15,verbose=1)
print(np.mean(model1.predict(xtrain),axis=0))


Epoch 1/15
5000/5000 [==============================] - 58s 12ms/step - loss: 0.1457
Epoch 2/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.1057
Epoch 3/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0952
Epoch 4/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0876
Epoch 5/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0811
Epoch 6/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0745
Epoch 7/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0702
Epoch 8/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0626
Epoch 9/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0574
Epoch 10/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0556
Epoch 11/15
5000/5000 [==============================] - 14s 3ms/step - loss: 0.0570
Epoch 12/15
5000/5000 [==============================] - 14s 3ms/step - l

In [24]:
Pmat[1.0]

[(0.9, 0.49334367044681626),
 (1.0, 0.4476271431557824),
 (1.1, 0.4102770904475874)]